In [6]:
#Importing Libraries

import numpy as np
import cvxpy as cvx
import pandas as pd
from datetime import datetime
import pandas_datareader.data as pdr

## Optimization with cvxpy

http://www.cvxpy.org/

Practice using cvxpy to solve a simple optimization problem. Find the optimal weights on a m-asset portfolio given the covariance matrix of the returns of m-stocks . Create a function that takes in these values as arguments and returns the vector of optimal weights.


## Hints

### x vector
To create a vector of M variables $\mathbf{x} = \begin{bmatrix}
x_1 &...& x_M
\end{bmatrix}
$
we can use `cvx.Variable(m)`

### covariance matrix
If we have $m$ stock series, the covariance matrix is an $m \times m$ matrix containing the covariance between each pair of stocks.  We can use [numpy.cov](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html) to get the covariance.  We give it a 2D array in which each row is a stock series, and each column is an observation at the same period of time.

The covariance matrix $\mathbf{P} = 
\begin{bmatrix}
\sigma^2_{1,1} & ... & \sigma^2_{1,m} \\ 
... & ... & ...\\
\sigma_{m,1} & ... & \sigma^2_{m,m}  \\
\end{bmatrix}$

### portfolio variance
We can write the portfolio variance $\sigma^2_p = \mathbf{x^T} \mathbf{P} \mathbf{x}$

Recall that the $\mathbf{x^T} \mathbf{P} \mathbf{x}$ is called the quadratic form.
We can use the cvxpy function `quad_form(x,P)` to get the quadratic form.

### optimization
So now that we have our objective function and constraints, we can solve for the values of $\mathbf{x}$.
cvxpy has the constructor `Problem(objective, constraints)`, which returns a `Problem` object.

The `Problem` object has a function solve(), which returns the minimum of the solution.  In this case, this is the minimum variance of the portfolio.

It also updates the vector $\mathbf{x}$.

We can check out the values of vector of weights x that gave the minimum portfolio variance by using `x.value`

In [7]:
def optimize_portfolio(returns):
    """
    Create a function that takes the return series of a set of stocks,The function 
    will minimize a combination of the portfolio variance.  
    The optimization will be constrained to be long only, and the weights should sum to one.
    
    Parameters
    ----------
    returns : numpy.ndarray
        2D array containing stock return series in each row.
        
    index_weights : numpy.ndarray
        1D numpy array containing weights of the index.
        

        
    Returns
    -------
    x : np.ndarray
        A numpy ndarray containing the weights of the stocks in the optimized portfolio
    """
    # TODO: Use cvxpy to determine the weights on the assets
    # that minimizes the  portfolio variance 
    
    # number of stocks m is number of rows of returns, and also number of index weights
    
    m = len(returns)
    #covariance matrix of returns
    cov = returns.cov()
    
    # x variables (to be found with optimization)
    x = cvx.Variable(m)
    
    #portfolio variance, in quadratic form
    portfolio_variance = quad_form(x,cov)   
    
    #objective function
    objective = cp.Minimize(portfolio_variance)
    
    #constraints
    constraints = [x >= 0, sum(x) == 1]

    #use cvxpy to solve the objective
    prob = cp.Problem(objective, constraints)
    prob.solve()
    x_values = x.value

    return x_values

In [9]:
#Stock_Data_Extraction

tickers = ["ASIANPAINT.NS","ADANIPORTS.NS","AXISBANK.NS","BAJAJ-AUTO.NS"]

start= datetime(2018,1,1)
end = datetime.today()

#Creeating a dictionary to store data for all stocks.
ohlc_daily = {}
for i in range(len(tickers)):
    ohlc_daily[tickers[i]] = pdr.get_data_yahoo(tickers[i],start = start , end = end)

#Creating a dataframe to store daily_returns of all stocks
returns = pd.DataFrame()
for ticker in tickers:
    returns[ticker] = (ohlc_daily[ticker]['Adj Close'].pct_change())
returns = returns.dropna(axis=0)

#Applying Optimization function to get min_variance_weights
min_var_weights = optimize_portfolio(returns.T)
# print(min_var_weights)

NameError: name 'quad_form' is not defined